In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
#from konlpy.tag import mecab
from gensim.models import Word2Vec
from scipy.spatial import distance
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer

## Data preprocessing

In [2]:
raw_df = pd.read_csv('../Data/data_for_use/theme_contents_200_final.csv', encoding = 'utf-8', index_col=0)
raw_df.dropna(inplace=True)

In [3]:
raw_df.head()

,theme,contents
0,2차전지,신종 코로나바이러스 감염증(코로나19) 사태로 인한 급락장에서 10대 그룹 시가총...
1,2차전지,2차전지·디스플레이 공정 장비 전문기업 나인테크가 기업인수목적회사(SPAC·스팩) ...
2,2차전지,2차전지·디스플레이 공정 장비 전문기업 나인테크가 교보7호스팩과 합병을 통한 코스...
3,2차전지,'울산형 뉴딜사업' 추진 등 코로나19 이후 지역경제 살리기에 나선 울산에 반가운...
4,2차전지,코스닥 2차 전지 소재 기업 에코프로비엠이 압도적 기술력을 바탕으로 가파른 성장 ...


In [4]:
# only leave korean chars

raw_df = raw_df[['theme', 'contents']]
raw_df['contents'] = raw_df['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
raw_df.head()

,theme,contents
0,2차전지,신종 코로나바이러스 감염증코로나 사태로 인한 급락장에서 대 그룹 시가총액 반등을 ...
1,2차전지,차전지디스플레이 공정 장비 전문기업 나인테크가 기업인수목적회사스팩 합병을 통해 코스...
2,2차전지,차전지디스플레이 공정 장비 전문기업 나인테크가 교보호스팩과 합병을 통한 코스닥 상...
3,2차전지,울산형 뉴딜사업 추진 등 코로나 이후 지역경제 살리기에 나선 울산에 반가운 소식이...
4,2차전지,코스닥 차 전지 소재 기업 에코프로비엠이 압도적 기술력을 바탕으로 가파른 성장 추...


In [5]:
# tokenize data using kobert tokenizer

tok_path = get_tokenizer()
sp  = SentencepieceTokenizer(tok_path)

raw_tokenized_data = []
for content in tqdm(raw_df['contents']):
    token = sp(content)
    raw_tokenized_data.append(token)

  1%|          | 230/33600 [00:00<00:14, 2296.08it/s]

using cached model


100%|██████████| 33600/33600 [00:17<00:00, 1933.35it/s]


In [6]:
# make train_data

train_data = raw_df.copy()
train_data['tokens'] = pd.Series(raw_tokenized_data)

In [7]:
train_data.head()

,theme,contents,tokens
0,2차전지,신종 코로나바이러스 감염증코로나 사태로 인한 급락장에서 대 그룹 시가총액 반등을 ...,"[▁신, 종, ▁코, 로, 나, 바이, 러스, ▁감염, 증, 코, 로, 나, ▁사태..."
1,2차전지,차전지디스플레이 공정 장비 전문기업 나인테크가 기업인수목적회사스팩 합병을 통해 코스...,"[▁, 차전, 지, 디스플레이, ▁공정, ▁장비, ▁전문, 기업, ▁나, 인, 테크..."
2,2차전지,차전지디스플레이 공정 장비 전문기업 나인테크가 교보호스팩과 합병을 통한 코스닥 상...,"[▁, 차전, 지, 디스플레이, ▁공정, ▁장비, ▁전문, 기업, ▁나, 인, 테크..."
3,2차전지,울산형 뉴딜사업 추진 등 코로나 이후 지역경제 살리기에 나선 울산에 반가운 소식이...,"[▁울산, 형, ▁뉴, 딜, 사업, ▁추진, ▁등, ▁코, 로, 나, ▁이후, ▁지..."
4,2차전지,코스닥 차 전지 소재 기업 에코프로비엠이 압도적 기술력을 바탕으로 가파른 성장 추...,"[▁코스닥, ▁차, ▁전, 지, ▁소재, ▁기업, ▁, 에, 코, 프로, 비, 엠,..."


## Model training

In [8]:
news_tokens = train_data['tokens'].tolist()  # data input as list

# params
v_dimension = 300
v_window = 8

model = Word2Vec(sentences = news_tokens, size = v_dimension, window = v_window, min_count = 5, workers = 4, sg = 0)

In [9]:
model.wv.vectors.shape

(7287, 300)

In [10]:
# save model
model.save("word2vec.model")

## Get theme vector representations

In [11]:
# get sentence vectors without normalization

def without_normal(tokens):
    vectors = []
    for token in tokens:
        init_v = np.array([0.0]*v_dimension)
        for word in token:
            word_vectors = model.wv
            if word in word_vectors.vocab:
                v = model.wv[word]
                init_v = init_v + v
        vectors.append(init_v)
    
    frame = { 'themes': train_data['theme'].tolist(), 'vectors': vectors }
    result = pd.DataFrame(frame) 
    
    return result

In [12]:
# get theme vectors without normalization

def theme_without_normal(news_df):
    theme_list = []
    vector_list = []
    for theme in news_df['themes'].unique():
        temp_df = news_df.loc[news_df['themes'] == theme]
        add_v = np.array([0.0]*v_dimension)
        for vec in temp_df['vectors']:
            add_v  = add_v + vec
        theme_list.append(theme)
        vector_list.append(add_v)
        
    frame = { 'themes': theme_list, 'vectors': vector_list }
    result = pd.DataFrame(frame)
    
    return result

In [13]:
news_vectors_df = without_normal(train_data['tokens'])

In [14]:
news_vectors_df.head()

,themes,vectors
0,2차전지,"[-31.791948940604925, 29.96601002267562, -4.72..."
1,2차전지,"[-24.93766422616318, 88.43383780308068, 40.867..."
2,2차전지,"[-11.37514140130952, 97.36546522052959, 36.757..."
3,2차전지,"[-231.28868790529668, 314.5986641594209, 26.12..."
4,2차전지,"[-3.1519764894619584, 215.4446411251556, -17.0..."


In [15]:
theme_vectors_df = theme_without_normal(news_vectors_df)

In [16]:
theme_vectors_df.head()

,themes,vectors
0,2차전지,"[-7096.778220521286, 12871.226797353534, 2587...."
1,3D프린터,"[10903.843000196677, 18503.521148866625, -3156..."
2,4대강,"[-8006.077763321286, 7393.415823798161, -24043..."
3,5G,"[-4483.314872191055, 15860.959652718622, 10811..."
4,AR,"[4912.215373729239, 17996.481813186198, 649.44..."


In [17]:
# convert array to list for output file
theme_vectors_df['vectors'] = theme_vectors_df['vectors'].apply(lambda x: x.tolist())
theme_vectors_df.to_csv('theme_vectors_xnorm.csv')